In [2]:
import numpy as np
from datetime import datetime, date, timedelta
import pandas as pd

In [3]:
class bonds:
        
    def __init__(self,
                 N : np.float,
                 tf : np.float,
                 tr : np.float,
                 d_emis : np.str,
                 d_jouis : np.str,
                 d_ech : np.str,
                 d_val : np.str):  
    
        self.N = N
        self.tf = tf
        self.tr = tr
        self.d_emis = datetime.strptime(d_emis, '%d/%m/%Y').date()
        self.d_jouis = datetime.strptime(d_jouis, '%d/%m/%Y').date()
        self.d_ech = datetime.strptime(d_ech, '%d/%m/%Y').date()
        self.d_val = datetime.strptime(d_val, '%d/%m/%Y').date()
           
        "sortie"
        
        self.dirty_price = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[0]
        self.coupon_couru = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[1]
        self.clean_price = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[2]
        self.coupon_recu = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[3]
        self.duration = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[4]
        self.sensibilite = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[5]
        self.convexite = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[6]
    
    def calculator(self,N,tf,tr,d_emis,d_jouis,d_ech,d_val): #7 parametres dont 4 dates
        
        "calcul intermediaire"
        
        # maturite residuelle et initiale
        self.mr = (self.d_ech - self.d_val).days
        self.mi = (self.d_ech - self.d_emis).days
        
        # date du dernier coupon (ddc) et date du prochain coupon (dpc)
        ddc = self.d_jouis
        dpc = ddc.replace(year = ddc.year+1)
        while dpc < self.d_val:
            dpc = dpc.replace(year= dpc.year +1)
        self.ddc = dpc.replace(year = dpc.year-1)
        self.dpc = dpc
        
        # Nombre de jours exact de l’année écoulée ???????
        self.A = 365
        
        # nombre de jours entre date de valorisation et date du prochain coupon 
        self.nj = (self.dpc - self.d_val).days 
        
        # nombre de coupon restant avant la maturité
        self.n = (self.d_ech.year - self.dpc.year) + 1
        
        # nombre de coupon deja recu avant la date de valorisation
        self.nc = (self.ddc.year - self.d_jouis.year) + 1
        
        # Date du premier coupon ??
        # d_pc = "date_jouissnce+ frequence"???

        if self.mi <= 365 : #maturite initiale inferieure a 1an

            C = N * tf * self.mi / 360
            Cr = C*self.nc
            P = (N + C) / (1+ tr*self.mr /360)
            Cc = N*tf* (self.mi-self.mr) / 360
            
            
            # Duration et sensiblite
            num = 0
            D = 0
            for i in range(1,self.n+1):
                num = num + (self.nj/self.A + i-1)*C
            num = num + (self.nj/self.A + self.n-1) * N
            D = num/ P
            S = D/(1+self.tr)
                    
            # Convexite
            Co = 0
            for i in range(1,self.n+1):
                num = num + C*(self.nj/self.A + i-1)* (self.nj/self.A + i) / (1+self.tr)**(self.nj/self.A + i-1)
            num = num + N * (self.nj/self.A + self.n-1)* (self.nj/self.A + self.n) / (1+self.tr)**(self.nj/self.A + self.n-1)
            Co = num/ (P*(1+self.tr)**2)              

        else:  #maturite initiale superieure a 1an

            if self.mr <= 365: #maturite residuelle inferieure a 1an

                    C = N*tf 
                    Cr = C*self.nc
                    P = (N + C) / (1+ tr*self.mr/360)
                    Cc = N*tf* (self.d_val-self.ddc).days / self.A
                    
                    # Duration et sensiblite
                    num = 0
                    D = 0
                    for i in range(1,self.n+1):
                        num = num + (self.nj/self.A + i-1)*C
                    num = num + (self.nj/self.A + self.n-1) * N
                    D = num/ P
                    S = D/(1+self.tr)
                    
                    # Convexite
                    Co = 0
                    for i in range(1,self.n+1):
                        num = num + C*(self.nj/self.A + i-1)* (self.nj/self.A + i) / (1+self.tr)**(self.nj/self.A + i-1)
                    num = num + N * (self.nj/self.A + self.n-1)* (self.nj/self.A + self.n) / (1+self.tr)**(self.nj/self.A + self.n-1)
                    Co = num/ (P*(1+self.tr)**2)

            else: # maturite residuelle superieure a 1an

                    C = N*tf
                    Cr = C*self.nc
                    Cc = N*tf* (self.d_val-self.ddc).days / self.A
                    s = 0
                    for i in range(1,self.n+1): # n compris
                        s = s + C/ (1+tr)**(i-1)
                    P = ( s + N/ (1+tr)**(self.n-1))  / (1+tr)**(self.nj / self.A)
                    
                    # Duration et sensiblite
                    num = 0
                    D = 0
                    for i in range(1,self.n+1):
                        num = num + C *( self.nj/self.A + i-1)
                    num = num + N*( self.nj/self.A + self.n-1)
                    D = num/ P
                    S = D/(1+self.tr)
                    
                    # Convexite
                    Co = 0
                    for i in range(1,self.n+1):
                        num = num + C*(self.nj/self.A + i-1)* (self.nj/self.A + i) / (1+self.tr)**(self.nj/self.A + i-1)
                    num = num + N * (self.nj/self.A + self.n-1)* (self.nj/self.A + self.n) / (1+self.tr)**(self.nj/self.A + self.n-1)
                    Co = num/ (P*(1+self.tr)**2)

                        
        return  (round(P),round(Cc),round(P-Cc),round(Cr), round(D,2), round(S,2), round(Co,2))
    

## Example

In [4]:
N = 100000
tf = 0.05150000
tr = 0.019763
d_emis = '05/06/2006'
d_jouis = '05/06/2007'
d_ech = '05/06/2026'
d_val = "15/06/2021"
bond1 = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val)
print('dirty_price = ', bond1.dirty_price)
print('clean_price = ', bond1.clean_price)
print('duration = ', bond1.duration)
print('Sensibiliite = ', bond1.sensibilite)
print('Convexite = ', bond1.convexite)

dirty_price =  115031
clean_price =  114890
duration =  4.99
Sensibiliite =  4.89
Convexite =  30.07


## Real data

In [5]:
import os
os.getcwd()
os.chdir("C:\\Users\\Halem\\Downloads\\final")

In [6]:
pd.set_option("display.max_row",100)
pd.set_option("display.max_column",100)
bond = pd.read_excel("ptfbonds.xlsx") 

In [7]:
#bond.head()

In [8]:
data = pd.DataFrame( {'Short Name' : bond['Short Name'],'Clean_price' : 1000*bond['Clean Price'], 
                      'Price' : None, 'Ecart' : None , 'Duration' : bond['Dur.'], 'Dur_th' : None, 'Ecart_dur' : None ,
                     'Sensibilite' : bond['Mod Dur'], 'Sen_th' : None, 'Ecart_sens' : None ,
                     'Convexite' : bond['Cnvx.'], 'Con_th' : None, 'Ecart_con' : None } )

In [9]:
N = 100000
for i in range (44): 
    tf = bond['Cpn Rate'][i]/100
    tr = bond['Yield Mat'][i]/100
    d_emis = bond['Date emission'][i].strftime("%d/%m/%Y")
    d_jouis = bond['Jouissance'][i].strftime("%d/%m/%Y")
    d_ech = bond['Mat. Date'][i].strftime("%d/%m/%Y")
    d_val = "01/06/2021"
    
    data['Price'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).clean_price
    data['Ecart'][i] = data['Price'][i]-data['Clean_price'][i]
    
    data['Dur_th'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).duration
    data['Ecart_dur'][i] = data['Dur_th'][i]-data['Duration'][i]
    
    data['Sen_th'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).sensibilite
    data['Ecart_sens'][i] = data['Sen_th'][i]-data['Sensibilite'][i]
    
    data['Con_th'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).convexite
    data['Ecart_con'][i] = data['Con_th'][i]-data['Convexite'][i]


In [10]:
data.iloc[0:44,:]

,Short Name,Clean_price,Price,Ecart,Duration,Dur_th,Ecart_dur,Sensibilite,Sen_th,Ecart_sens,Convexite,Con_th,Ecart_con
0,MA0002007096,111832,111911,79,2.67,2.82,0.15,2.62,2.77,0.15,9.668,12.51,2.842
1,MA0002007641,116610,116679,69,3.84,4.18,0.34,3.77,4.1,0.33,19.068,23.25,4.182
2,MA0002008128,101404,101445,41,0.67,0.69,0.02,0.67,0.68,0.01,0.886,1.78,0.894
3,MA0002009936,108384,108427,43,3.5,3.77,0.27,3.44,3.7,0.26,15.676,19.46,3.784
4,MA0002010934,112970,113012,42,5.31,5.97,0.66,5.2,5.85,0.65,33.829,39.78,5.951
5,MA0002011114,102255,102306,51,0.87,0.89,0.02,0.86,0.88,0.02,1.482,2.47,0.988
6,MA0002012369,120616,120667,51,6.06,7.01,0.95,5.93,6.87,0.94,45.253,52.22,6.967
7,MA0002012922,107449,107513,64,2.02,2.11,0.09,1.99,2.08,0.09,6.101,8.24,2.139
8,MA0002013441,135677,135721,44,10.36,14.15,3.79,10.11,13.8,3.69,130.771,144.63,13.859
9,MA0002013797,114208,114236,28,7.72,9.4,1.68,7.55,9.19,1.64,71.254,80.55,9.296


In [11]:
"""
df1 = data['Ecart']
df1.to_csv("C:\\Users\\Halem\\Downloads\\final\\remi1.csv", header=True, sep='\n')

df2 = data['Ecart_dur']
df2.to_csv("C:\\Users\\Halem\\Downloads\\final\\remi2.csv", header=True, sep='\n')

df3 = data['Ecart_con']
df3.to_csv("C:\\Users\\Halem\\Downloads\\final\\remi3.csv", header=True, sep='\n')
"""

'\ndf1 = data[\'Ecart\']\ndf1.to_csv("C:\\Users\\Halem\\Downloads\\final\\remi1.csv", header=True, sep=\'\n\')\n\ndf2 = data[\'Ecart_dur\']\ndf2.to_csv("C:\\Users\\Halem\\Downloads\\final\\remi2.csv", header=True, sep=\'\n\')\n\ndf3 = data[\'Ecart_con\']\ndf3.to_csv("C:\\Users\\Halem\\Downloads\\final\\remi3.csv", header=True, sep=\'\n\')\n'

## Valorisation

In [12]:
pd.set_option("display.max_row",10)
pd.set_option("display.max_column",100)
bond = pd.read_excel("ptfbonds.xlsx") 

In [13]:
data2 = pd.DataFrame( {'Nominal' : 100000 ,'Quantite' : bond['Quantity'],
                      'Taux facial' : bond['Cpn Rate'], 'Date emission' : bond['Date emission'], 
                      'Date jouissance' : bond['Jouissance'], 'Date maturite' : bond['Mat. Date'], 
                      'Prix dirty' : None, 'Prix clean' : None,
                      'Coupon couru' : None  } )
data2
N = 100000
for i in range (10): 
    tf = bond['Cpn Rate'][i]/100
    tr = bond['Yield Mat'][i]/100
    d_emis = bond['Date emission'][i].strftime("%d/%m/%Y")
    d_jouis = bond['Jouissance'][i].strftime("%d/%m/%Y")
    d_ech = bond['Mat. Date'][i].strftime("%d/%m/%Y")
    d_val = "01/06/2021"
    
    data2['Prix clean'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).clean_price
    data2['Prix dirty'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).dirty_price
    data2['Coupon couru'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).coupon_couru
    
data2.head()

C:\Users\Halem\anaconda3_best\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\Halem\anaconda3_best\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Halem\anaconda3_best\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Nominal,Quantite,Taux facial,Date emission,Date jouissance,Date maturite,Prix dirty,Prix clean,Coupon couru
0,100000,7445,6.1,2004-04-05,2004-04-05,2024-04-05 00:00:00,112864,111911,953
1,100000,8440,5.95,2005-10-03,2005-10-03,2025-10-03 00:00:00,120608,116679,3929
2,100000,637,3.65,2007-02-05,2007-02-05,2022-02-05 00:00:00,102605,101445,1160
3,100000,1835,4.2,2010-03-01,2010-03-01,2025-03-01 00:00:00,109486,108427,1059
4,100000,1780,4.4,2012-01-02,2012-04-19,2027-04-19 00:00:00,113530,113012,518


In [14]:
data3 = pd.DataFrame( {'Clean_price' : data2['Prix clean'], 
                       'Duration' : bond['Dur.'], 'Sensibilite' : bond['Mod Dur'],
                     'Convexite' : bond['Cnvx.'] } )
data3


,Clean_price,Duration,Sensibilite,Convexite
0,111911,2.67,2.62,9.668
1,116679,3.84,3.77,19.068
2,101445,0.67,0.67,0.886
3,108427,3.5,3.44,15.676
4,113012,5.31,5.2,33.829
...,...,...,...,...
48,None,3.67,3.6,17.239
49,None,5.76,5.64,40.149
50,None,7.8,7.63,72.751
51,None,13.61,13.24,235.964
